<a href="https://colab.research.google.com/github/saurabhsingh1411/imdb_dnn_keras/blob/master/Imdb_dataset_DNN_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Lib and datsets

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_datasets as tfds

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
dataset=pd.read_csv('IMDB Dataset.csv')

In [3]:
dataset['sentiment'].value_counts().plot

In [4]:
negative = len(dataset[dataset['sentiment']=='positive'])
positive = len(dataset) - negative

print('Positive reviews are {} and negative reviews are {} of total {} '.format(positive,negative,len(dataset)))

Positive reviews are 25000 and negative reviews are 25000 of total 50000 


In [5]:
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
le=LabelEncoder()

X_train,X_test,y_train,y_test=train_test_split(dataset['review'].values,dataset['sentiment'].values,test_size=0.2)

In [7]:
y_train=le.fit_transform(y_train)
y_test=le.fit_transform(y_test)

In [8]:
y_test=np.array(y_test)

In [9]:
print(type(y_train))
print(type(y_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


## Preprocessing Text

In [10]:
tokenizer=Tokenizer(num_words=10000,oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
word_index=tokenizer.word_index

training_seq=tokenizer.texts_to_sequences(X_train)

testing_seq=tokenizer.texts_to_sequences(X_test)

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

training_pad_seq=pad_sequences(training_seq,maxlen=200,truncating='post',padding='pre')
testing_pad_seq=pad_sequences(testing_seq,maxlen=200,truncating='post',padding='pre')

In [12]:
print('total unique words',len(word_index))

total unique words 111984


## Model Buiulding

In [13]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(len(word_index) + 1,200,input_length=200,
                            trainable=False),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Dense(256,activation = 'relu',),
                             tf.keras.layers.Dense(128,activation = 'relu'),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Dense(1,activation = tf.nn.sigmoid)])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 200)          22397000  
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               135680    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               33024     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1

In [15]:
model.compile(loss = tf.keras.losses.BinaryCrossentropy() , optimizer='Adam' , metrics = 'accuracy')

In [ ]:
history = model.fit(training_pad_seq,y_train,epochs = 30 ,validation_data=(testing_pad_seq,y_test))

Epoch 1/30
1250/1250 [==============================] - 237s 189ms/step - loss: 0.6342 - accuracy: 0.6306 - val_loss: 0.5894 - val_accuracy: 0.6903
Epoch 2/30
 715/1250 [================>.............] - ETA: 1:34 - loss: 0.5701 - accuracy: 0.7100

In [38]:
print(type(training_pad_seq))
print(type(y_train))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [39]:
print(type(testing_pad_seq))
print(type(y_test))

<class 'numpy.ndarray'>
<class 'sklearn.preprocessing._label.LabelEncoder'>
